In [1]:
from tensorflow.keras.callbacks import EarlyStopping, History, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, AveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Model, load_model
#from google.colab import drive
from datetime import datetime
import tensorflow as tf
import numpy as np
import warnings
import os

#drive.mount('/content/drive')
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print("Los dispositivos encontrados son: ", gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "GPUs físicos,", len(logical_gpus), "GPUs lógicos")
  except RuntimeError as e:
    print(e)
print("----------------------------------------------------- \n")

Los dispositivos encontrados son:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 GPUs físicos, 1 GPUs lógicos
----------------------------------------------------- 



## Generación del modelo

In [3]:
def input_section(input_shape):
    #First section of the input block
    conv_1 = Conv2D(filters = 32, kernel_size = (3, 3), strides = (2,2), activation = "relu")(input_shape)
    conv_2 = Conv2D(filters = 32, kernel_size = (3, 3), activation = "relu")(conv_1)
    conv_3 = Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', activation = "relu")(conv_2)
    conv_4 = Conv2D(filters = 96, kernel_size = (3, 3), strides = (2,2), activation = "relu")(conv_3)
    maxp_1 = MaxPooling2D(pool_size=(3, 3), strides= (2,2))(conv_3)  
    conc_1 = concatenate([maxp_1, conv_4], axis=3)
    
    #Second section of the input block
    conv_5 = Conv2D(filters = 64, kernel_size = (1, 1), padding = 'same',activation = "relu")(conc_1)
    conv_6 = Conv2D(filters = 64, kernel_size = (1, 1), padding = 'same', activation = "relu")(conc_1)
    conv_7 = Conv2D(filters = 96, kernel_size = (3, 3), activation = "relu")(conv_6)
    conv_8 = Conv2D(filters = 64, kernel_size = (7, 1), padding = 'same', activation = "relu")(conv_6)
    conv_9 = Conv2D(filters = 64, kernel_size = (1, 7), padding = 'same', activation = "relu")(conv_8)
    conv_10 = Conv2D(filters = 96, kernel_size = (3, 3), activation = "relu")(conv_9)
    conc_2 = concatenate([conv_7, conv_10], axis=3)
    
    #Third section of the input block
    maxp_2 = MaxPooling2D(pool_size=(3, 3),strides= (2,2))(conc_2)
    conv_11 = Conv2D(filters = 192, kernel_size = (3, 3),strides = (2,2), activation = "relu")(conc_2)
    conc_3 = concatenate([conv_11, maxp_2], axis=3)
    
    return conc_3

In [4]:
def inception_A_block(input_shape):
    conv_1 = Conv2D(filters = 64, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_2 = Conv2D(filters = 64, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_3 = Conv2D(filters = 96, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    avpo_1 = AveragePooling2D(pool_size=(1, 1), padding='same')(input_shape)
    
    conv_4 = Conv2D(filters = 96, kernel_size = (3, 3), padding = 'same', activation = "relu")(conv_1)
    conv_5 = Conv2D(filters = 96, kernel_size = (3, 3), padding = 'same', activation = "relu")(conv_2)
    conv_6 = Conv2D(filters = 96, kernel_size = (1, 1), padding = 'same', activation = "relu")(avpo_1)
    
    conv_7 = Conv2D(filters = 96, kernel_size = (3, 3), padding = 'same', activation = "relu")(conv_4)
    conc_1 = concatenate([conv_7,conv_5,conv_3,conv_6], axis = 3)
    
    return conc_1

In [5]:
def reduction_A_block(input_shape):
    conv_1 = Conv2D(filters = 192, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_2 = Conv2D(filters = 224, kernel_size = (3, 3), padding = 'same', activation = "relu")(conv_1)
    conv_3 = Conv2D(filters = 384, kernel_size = (3, 3), strides = (2,2), activation = "relu")(input_shape)

    maxp_1 = MaxPooling2D(pool_size=(3, 3),strides= (2,2))(input_shape)
    conv_4 = Conv2D(filters = 256, kernel_size = (3, 3), strides = (2,2), activation = "relu")(conv_2)
    
    conc_1 = concatenate([conv_4,conv_3,maxp_1], axis = 3)
    
    return conc_1

In [6]:
def inception_B_block(input_shape):
    conv_1 = Conv2D(filters = 192, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_2 = Conv2D(filters = 192, kernel_size = (1, 7), padding = 'same', activation = "relu")(conv_1)
    conv_3 = Conv2D(filters = 224, kernel_size = (7, 1), padding = 'same', activation = "relu")(conv_2)
    conv_4 = Conv2D(filters = 224, kernel_size = (1, 7), padding = 'same', activation = "relu")(conv_3)
    conv_5 = Conv2D(filters = 256, kernel_size = (7, 1), padding = 'same', activation = "relu")(conv_4)
    
    conv_6 = Conv2D(filters = 192, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_7 = Conv2D(filters = 224, kernel_size = (1, 7), padding = 'same', activation = "relu")(conv_6)
    conv_8 = Conv2D(filters = 256, kernel_size = (1, 7), padding = 'same', activation = "relu")(conv_7)
    
    avpo_1 = AveragePooling2D(pool_size=(1, 1), padding='same')(input_shape)
    conv_9 = Conv2D(filters = 128, kernel_size = (1, 1), padding = 'same', activation = "relu")(avpo_1)
    
    conv_10 = Conv2D(filters = 384, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    
    conc_1 = concatenate([conv_5,conv_8,conv_9,conv_10], axis = 3)
    
    return conc_1

In [7]:
def reduction_B_block(input_shape):
    conv_1 = Conv2D(filters = 256, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_2 = Conv2D(filters = 256, kernel_size = (1, 7), padding = 'same', activation = "relu")(conv_1)
    conv_3 = Conv2D(filters = 320, kernel_size = (7, 1), padding = 'same', activation = "relu")(conv_2)
    conv_4 = Conv2D(filters = 320, kernel_size = (3, 3), strides = (2, 2), activation = "relu")(conv_3)

    conv_5 = Conv2D(filters = 192, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_6 = Conv2D(filters = 192, kernel_size = (3, 3), strides = (2, 2), activation = "relu")(conv_5)
    
    maxp_1 = MaxPooling2D(pool_size=(3, 3),strides= (2,2))(input_shape)

    
    conc_1 = concatenate([conv_4,conv_6,maxp_1], axis = 3)
    
    return conc_1

In [8]:
def inception_C_block(input_shape):
    conv_1 = Conv2D(filters = 384, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_2 = Conv2D(filters = 448, kernel_size = (1, 3), padding = 'same', activation = "relu")(conv_1)
    conv_3 = Conv2D(filters = 512, kernel_size = (3, 1), padding = 'same', activation = "relu")(conv_2)
    conv_4 = Conv2D(filters = 256, kernel_size = (1, 3), padding = 'same', activation = "relu")(conv_3)
    conv_5 = Conv2D(filters = 256, kernel_size = (3, 1), padding = 'same', activation = "relu")(conv_3)
    
    conv_6 = Conv2D(filters = 384, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    conv_7 = Conv2D(filters = 256, kernel_size = (3, 1), padding = 'same', activation = "relu")(conv_6)
    conv_8 = Conv2D(filters = 256, kernel_size = (1, 3), padding = 'same', activation = "relu")(conv_6)
    
    avpo_1 = AveragePooling2D(pool_size=(1, 1), padding='same')(input_shape)
    conv_9 = Conv2D(filters = 256, kernel_size = (1, 1), padding = 'same', activation = "relu")(avpo_1)
    
    conv_10 = Conv2D(filters = 256, kernel_size = (1, 1), padding = 'same', activation = "relu")(input_shape)
    
    conc_1 = concatenate([conv_5,conv_4,conv_7,conv_8,conv_10,conv_9], axis = 3)
    
    return conc_1

In [10]:
tam = (300,300,3)
input_shape = Input(tam)

#Input inception block
inp = input_section(input_shape)

#Inception block-A (x4)
A1 = inception_A_block(inp)
A2 = inception_A_block(A1)
A3 = inception_A_block(A2)     
A4 = inception_A_block(A3)

#Reduction model A
AR = reduction_A_block(A4)
AR = BatchNormalization()(AR)
AR = MaxPooling2D(pool_size=(3, 3),strides= (2,2))(AR)

#Inception block-B (x7)
B1 = inception_B_block(AR)
B2 = inception_B_block(B1)
B3 = inception_B_block(B2)
B4 = inception_B_block(B3)
B5 = inception_B_block(B4)
B6 = inception_B_block(B5)
B7 = inception_B_block(B6)
    

#Reduction model B
BR = reduction_B_block(B7)
BR = BatchNormalization()(BR)
BR = MaxPooling2D(pool_size=(3, 3),strides= (2,2))(BR)

#Inception block-C (x3)
C1 = inception_C_block(BR)
C2 = inception_C_block(C1)
C3 = inception_C_block(C2)
C3 = BatchNormalization()(C3)

#Detection part

GLB = GlobalAveragePooling2D()(C3)

FC = Dense(2048)(GLB)
DP = Dropout(0.5)(FC)

y_pred = Dense(1, activation= "sigmoid")(DP)


#Creating model
model = Model(inputs = input_shape, outputs= y_pred)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv2d_119 (Conv2D)             (None, 149, 149, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_120 (Conv2D)             (None, 147, 147, 32) 9248        conv2d_119[0][0]                 
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 147, 147, 64) 18496       conv2d_120[0][0]                 
______________________________________________________________________________________________

## Acoplamiento de los datos

In [12]:
input_dir= "/home/revientaelp/Documentos/Bases de datos/Sharon/patch_m300/entA"
target_dir= "/home/revientaelp/Documentos/Bases de datos/Sharon/patch_m300/valA"

generador_ent= ImageDataGenerator(rescale= 1/255.0)
generador_val= ImageDataGenerator(rescale= 1/255.0)


ent_gen= generador_ent.flow_from_directory(input_dir, 
                                           target_size=(300, 300),
                                           class_mode= 'binary',
                                           batch_size= 16,
                                           color_mode= "rgb",
                                           shuffle= True,
                                           )

val_gen= generador_val.flow_from_directory(target_dir, 
                                           target_size=(300, 300),
                                           class_mode= 'binary',
                                           color_mode= "rgb",
                                           batch_size= 16,
                                           shuffle= True,
                                           )

Found 15954 images belonging to 2 classes.
Found 1194 images belonging to 2 classes.


In [13]:
def entrenamient_modelo(train_gen, val_gen, DIR_save, epocas= 30):

    #class_weights = {0:1., 1:1.5, 2:1.5, 3:4, 4:4}

    #Generamos los callback necesarios
    checkpoint= ModelCheckpoint(DIR_save+ "01092021_A1.hdf5", monitor= 'val_loss', save_best_only= True, mode= 'min', save_weights_only= False)
    #reduceLROnPlat= ReduceLROnPlateau(monitor= 'val_loss', factor= 0.8, patience= 15, min_delta= 0.001, cooldown= 10, min_lr= 0.00001)
    #tensorboard_callback = TensorBoard()#log_dir= datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks= [EarlyStopping(patience= 30,  monitor='val_loss'), checkpoint]#, tensorboard_callback]
    metrica= [tf.keras.metrics.AUC(),
              tf.keras.metrics.BinaryAccuracy(),
              tf.keras.metrics.Recall(),
              tf.keras.metrics.Precision()]

    #Compilamos con la funcion perdida y el optimizador que utilizaremos
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.00001), loss = 'binary_crossentropy', metrics= metrica)
    history= model.fit(train_gen, batch_size= 16, epochs=epocas, validation_data=val_gen, callbacks=callbacks)

def carga_mod(DIR):
    return tf.keras.models.load_model(DIR)

In [14]:
DIR_save_best= '/home/revientaelp/Escritorio/'
#DIR_save_best= '/content/drive/MyDrive/Modelos guardados/Exudados_SGD/'
#DIR_load_best= '/content/drive/MyDrive/Modelos guardados/Clasificacion_exudados_SGD_IV4/06082021_400.hdf5'

#model= carga_mod(DIR_load_best)
entrenamient_modelo(ent_gen, val_gen, DIR_save_best, 400)

Epoch 1/400
998/998 [==============================] - 399s 400ms/step - loss: 0.3906 - auc: 0.8733 - binary_accuracy: 0.8320 - recall: 0.8926 - precision: 0.8595 - val_loss: 0.3505 - val_auc: 0.5946 - val_binary_accuracy: 0.8819 - val_recall: 0.9915 - val_precision: 0.8880
Epoch 2/400
998/998 [==============================] - 304s 305ms/step - loss: 0.3068 - auc: 0.9046 - binary_accuracy: 0.8865 - recall: 0.9628 - precision: 0.8778 - val_loss: 0.3690 - val_auc: 0.6235 - val_binary_accuracy: 0.8693 - val_recall: 0.9574 - val_precision: 0.9011
Epoch 3/400
998/998 [==============================] - 310s 310ms/step - loss: 0.2957 - auc: 0.9088 - binary_accuracy: 0.8936 - recall: 0.9682 - precision: 0.8827 - val_loss: 0.3851 - val_auc: 0.5336 - val_binary_accuracy: 0.8719 - val_recall: 0.9858 - val_precision: 0.8830
Epoch 4/400
998/998 [==============================] - 309s 310ms/step - loss: 0.2872 - auc: 0.9138 - binary_accuracy: 0.8990 - recall: 0.9743 - precision: 0.8850 - val_loss: 